In [360]:
import os 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import date


loading in data 


In [361]:
os.getcwd()


'c:\\Users\\Olivia_Pinney\\Documents\\UChicago_Materials\\Data_Viz\\CAPP30239_FA22\\final\\data\\extra_data'

In [362]:
data_path_iigh={
  "history_profiles":"chicago-police-data/data/unified_data/profiles/final-profiles.csv",
  "history_roster":"chicago-police-data/data/unified_data/roster/roster_1936-2017_2017-04.csv",
  "history_unit":"chicago-police-data/data/unified_data/unit-history/unit-history.csv", 
  "trr_overall":"chicago-police-data/data/unified_data/TRR/TRR-main_2004-2016_2016-09.csv",
  "trr_subjects":"chicago-police-data/data/unified_data/TRR/TRR-subjects_2004-2016_2016-09.csv",
  "trr_officers":"chicago-police-data/data/unified_data/TRR/TRR-officers_2004-2016_2016-09.csv",
  "trr_actions":"chicago-police-data/data/unified_data/TRR/TRR-actions-responses_2004-2016_2016-09.csv",
  "complaints":"chicago-police-data/data/unified_data/complaints/complaints-accused.csv", 
  "complaints_detailed":"chicago-police-data/data/unified_data/complaints/complaints-complaints.csv",
  "complaint_types":"chicago-police-data/data/context_data/Complaint Categories.csv", 
}

# Department Roster

In [363]:
history_roster=pd.read_csv("roster_1936-2017_2017-04.csv")
# history_unit=pd.read_csv(data_path_iigh["history_unit"]) 

In [364]:
df=history_roster 
df["resignation_date"]=df["resignation_date"].fillna("2999-01-01")
df["resignation_year"]=df["resignation_date"].apply(lambda x: int(str(x)[0:4]))

df["appointed_date"]=df["appointed_date"].fillna("1000-01-01")
df["appointed_year"]=df["appointed_date"].apply(lambda x: int(str(x)[0:4]))

df=df[df["appointed_year"]<2015] #officer in 2014
df=df[df["resignation_year"]>2013] #officer in 2014

#check that no one is duplicated
temp=df.UID.value_counts().reset_index()
problem_uids=temp[temp.UID>1]["index"]
print(problem_uids)
print(df.shape)
df=df[~(df["UID"].isin(problem_uids) & ~df["current_status"]>0)]
print(df.shape)


0    131785.0
1    131783.0
2    131787.0
3    131786.0
4    131788.0
5    131784.0
6    131789.0
Name: index, dtype: float64
(12965, 36)
(12958, 36)


In [365]:
df["unit_cat"]=np.where(df["unit_description"].str.contains("DISTRICT 0"),"DISTRICT",
    np.where(df["unit_description"].str.contains("DETECTIVE"),"DETECTIVE",
    np.where(df["unit_description"].str.contains("SPECIAL INVESTIGATIONS UNIT"),"DETECTIVE",
    np.where(df["unit_description"].str.contains("TRAFFIC"),"TRANSPORT",
    np.where(df["unit_description"].str.contains("TRANSPORT"),"TRANSPORT",
    np.where(df["unit_description"].str.contains("TRANSIT"),"TRANSPORT",    
    np.where(df["unit_description"].str.contains("AIRPORT"),"TRANSPORT",
    np.where(df["unit_description"].str.contains("GANG ENFORCEMENT"),"GANG ENFORCEMENT",
    "OTHER"
    ))))))))

In [366]:
temp=df[["unit_description","unit_cat"]].value_counts(sort=True)
temp=temp.reset_index()
#temp=temp.set_index("unit_description")
temp.columns=["unit_description","unit_cat","headcount"]
temp["p_w_alts_2014"]=0
temp["p_w_civ_all"]=0
temp["p_w_dept_all"]=0

temp.to_json("departments_basic.json",orient="records")


In [367]:
import json

with open("departments_basic.json") as file:
    data = json.load(file)

# Section of Complaints

In [368]:
#merge complaints
complaints=pd.read_csv("complaints-accused.csv")
complaints_detailed=pd.read_csv("complaints-complaints.csv")
complaint_types=pd.read_csv("Complaint Categories.csv")

cmp= complaints.merge(complaints_detailed[["cr_id", "complaint_date"]],on="cr_id",how="left")
cmp["code"]=cmp["complaint_code"]
complaint_types=complaint_types[~pd.isna(complaint_types["111"])]
complaint_types["code"]=complaint_types["111"]
cmp= cmp.merge(complaint_types,on="code",how="left")

C:\Users\Olivia_Pinney\AppData\Local\Temp\ipykernel_17176\2517323982.py:2: DtypeWarning: Columns (3,7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  complaints=pd.read_csv("complaints-accused.csv")
C:\Users\Olivia_Pinney\AppData\Local\Temp\ipykernel_17176\2517323982.py:3: DtypeWarning: Columns (0,5,6,7,8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  complaints_detailed=pd.read_csv("complaints-complaints.csv")


Need to add a section on the outcome of the complaints

In [369]:
#complaint results
comp_cat={"NAF":"No Affidavit", "EX":"Complaint Dismissed", "UN":"Complaint Dismissed","NS":"Insufficient Evidence","SU":"Complaint Accepted", "DIS":"Complaint Accepted","NC":"Missing Complaint Type","Missing":"Missing"}
#NC is missing

#outcome
#complaints.final_outcome.value_counts()

#less important:
dept_exclude=["OPERATION/PERSONNEL VIOLATION:  MISCELLANEOUS","MISCELLANEOUS", "OPERATION/PERSONNEL VIOLATION:  REPORTS - FAIL TO SUBMIT/IMPROPER", "OPERATION/PERSONNEL VIOLATION:  INVENTORY PROCEDURES","REPORTS - FAILED TO SUBMIT/IMPROPER","INVENTORY PROCEDURES","OPERATION/PERSONNEL VIOLATION: COMMUNICATION OPERATION PROCEDURES","COMMUNICATION OPERATIONS PROCEDURES","OPERATION/PERSONNEL VIOLATION:  VEHICLE LICENSING - CITY","MISUSE OF DEPARTMENT EQUIPMENT/SUPPLIES","INSUBORDINATION","OPERATION/PERSONNEL VIOLATION:  INSUBORDINATION","MEDICAL ROLL","OPERATION/PERSONNEL VIOLATION:  MEDICAL ROLL","OPERATION/PERSONNEL VIOLATION:  WEAPON/AMMUN./UNIFORM DEVIATION","FAIL TO OBTAIN A COMPLAINT REGISTER NUMBER","OPERATION/PERSONNEL VIOLATION: MISUSE OF DEPT. EQUIPMENT/SUPPLIES","OPERATION/PERSONNEL VIOLATION:  ABSENT WITHOUT PERMISSION","WEAPON/AMMUNITION/UNIFORM DEVIATION","SUPERVISOR RESPONSIBILITY:  FAIL TO OBTAIN COMPLAINT REGISTER NO.","SUPERVISOR RESPONSIBILITY:  MISCELLANEOUS","OPERATION/PERSONNEL VIOLATION:  LEAVING ASGMT (DIST/BEAT/SECT/CT)","ABSENT WITHOUT PERMISSION","OPERATION/PERSONNEL VIOLATION:  RESIDENCY","LEAVING ASSIGNMENT (DISTRICT, BEAT, SECTOR, COURT)","RESIDENCY","OPERATION/PERSONNEL VIOLATION:  COMPENSATORY TIME","VEHICLE LICENSING - CITY","OPERATION/PERSONNEL VIOLATION:  LATE FOR ROLL CALL/ASSIGNMENT/CT.","COMPENSATORY TIME","SUPVR. RESPONSIBILITY:  PROPER ACTION REVIEW/INSPECT- SUBORDINATE","OPERATION/PERSONNEL VIOLATION:  LUNCH/PERSONAL","LATE - ROLL CALL/ASSIGNMENT/COURT","LUNCH/PERSONAL VIOLATIONS","PROPER ACTION REVIEW/INSPECT - SUBORDINATE","EQUIPMENT IRREGULARITY"]


In [370]:
force=["EXCESSIVE FORCE","STRIKE","ASSAULT","BATTERY","PUSH","PULL","GRAB","CHOKED","KICKED","DRAGGED","TASER","SHOTS FIRED","COERCION - VIOLENCE","MURDER","MANSLAUGHTER","TAKE DOWN","INJURY","DEATH","FIREARM","WEAPON","PHYSICAL CONTACT"]
detain=["DETENTION","WARRANT","SEARCH","ARRESTEE","ARREST","LOCKUP","BONDING","BOOKING","PROCESSING","PRISONER'S PROPERTY", "PRISONERS PROPERTY"]
hate=["RACIAL","SEXUAL ORIENTATION","VERBAL ABUSE", "USE OF PROFANITY", "THREATS"]
crime=["COMMISSION OF CRIME", "DAMAGE","TRESPASSING", "BURGLARY", "THEFT", "FELONY","DRUGS","CONTR. SUB", "ROBBERY", "SHOPLIFTING","DOMESTIC","RAPE"]

#contains
citizen_cats={"EXCESSIVE FORCE":"EXCESSIVE FORCE","STRIKE":"EXCESSIVE FORCE","ASSAULT":"EXCESSIVE FORCE","BATTERY":"EXCESSIVE FORCE","PUSH":"EXCESSIVE FORCE","PULL":"EXCESSIVE FORCE","GRAB":"EXCESSIVE FORCE","INJURY":"EXCESSIVE FORCE","CHOKED":"EXCESSIVE FORCE","KICKED":"EXCESSIVE FORCE","DRAGGED":"EXCESSIVE FORCE","TASER":"EXCESSIVE FORCE","SHOTS FIRED":"EXCESSIVE FORCE","COERCION - VIOLENCE":"EXCESSIVE FORCE","MURDER/MANSLAUGHTER, ETC.":"EXCESSIVE FORCE","TAKE DOWN":"EXCESSIVE FORCE",
    "INJURY/DEATH":"INJURY/DEATH", 
    "UNNECESSARY PHYSICAL CONTACT":"UNNECESSARY PHYSICAL CONTACT",
    "FIREARM": "WEAPON USAGE", "WEAPON":"WEAPON USAGE",
    "EXCESSIVE DETENTION":"EXCESSIVE DETENTION",
    "WARRANT": "WARRANT / SEARCH VIOLATION","SEARCH":"SEARCH WITHOUT WARRANT",
    "ARRESTEE": "HANDLING OF ARREST / BOOKING", "ARREST/LOCKUP": "HANDLING OF ARREST / BOOKING", "BONDING/BOOKING/PROCESSING": "HANDLING OF ARREST / BOOKING","PRISONER'S PROPERTY":"HANDLING OF ARREST / BOOKING" , "PRISONERS PROPERTY":"HANDLING OF ARREST / BOOKING",
    "ILLEGAL ARREST": "ILLEGAL ARREST",
    "IMPROPER DETENTION":"UNLAWFUL DETENTION", "UNLAWFUL / EXCESSIVE INVESTIGATIVE DETENTION (WITNESS)": "UNLAWFUL DETENTION",
    "RACIAL": "RACIAL PROFILING / ABUSIVE LANGUAGE",
    "SEXUAL ORIENTATION":"SEXUAL ORIENTATION",
    "VERBAL ABUSE":"VERBAL ABUSE",  "USE OF PROFANITY": "VERBAL ABUSE", "THREATS": "VERBAL ABUSE",
    "DOMESTIC":"DOMESTIC ALTERCATION",
    "RAPE/SEX":"RAPE/SEX OFFENSES",
    "TRAFFIC": "TRAFFIC (NOT BRIBERY/EX-FORCE)",
    "NO ARREST": "NO ARREST",
    "COMMISSION OF CRIME":"CRIME", "DAMAGE/TRESPASSING PROPERTY":"CRIME", "BURGLARY":"CRIME", "THEFT":"CRIME", "OTHER FELONY":"CRIME","DRUGS/CONTR. SUB., POSSESSION OR SALE":"CRIME", "ROBBERY":"CRIME", "SHOPLIFTING":"CRIME",
    "ATTORNEY/RELATIVE PRIVILEGES":"COMMUNICATION","FIRST AMENDMENT":"COMMUNICATION",
    "MISCONDUCT DURING ISSUANCE OF CITATION":"OTHER","CIVIL SUIT":"OTHER", "IMPROPER PROCESSING/REPORTING/PROCEDURES":"OTHER","ESCAPE":"OTHER","MISCELLANEOUS":"OTHER","PARKING COMPLAINTS":"OTHER","VIOLATION (OTHER THAN D.U.I.) - ON DUTY":"OTHER","POLICE IMPERSONATOR - ADV SECTION USE ON CLOSING ONLY":"OTHER"}


In [371]:
cmp["civ_complaint_force"]=False
cmp["civ_complaint_detain"]=False
cmp["civ_complaint_hate"]=False
cmp["civ_complaint_crime"]=False

cmp["civ_complaint_flag"]=np.where(cmp["CITIZEN / DEPT"]=="CITIZEN",1,0)
cmp["civ_complaint_desc"]=np.where(cmp["civ_complaint_flag"],cmp.complaint_category,"")
cmp["civ_complaint_desc"]=np.where(cmp["civ_complaint_desc"].isna(),"Missing",cmp["civ_complaint_desc"])

for word in force:
    cmp["civ_complaint_force"]=np.where(cmp["civ_complaint_desc"].str.contains(word,case=False),True,cmp["civ_complaint_force"])
for word in detain:
    cmp["civ_complaint_detain"]=np.where(cmp["civ_complaint_desc"].str.contains(word,case=False),True,cmp["civ_complaint_detain"])
for word in hate:
    cmp["civ_complaint_hate"]=np.where(cmp["civ_complaint_desc"].str.contains(word,case=False),True,cmp["civ_complaint_hate"])
for word in crime:
    cmp["civ_complaint_crime"]=np.where(cmp["civ_complaint_desc"].str.contains(word,case=False),True,cmp["civ_complaint_crime"])

duplicate_words_detain=["EXCESSIVE FORCE:  ARRESTEE","EXCESSIVE FORCE:  NO ARREST"]
duplicate_words_force=["COMMISSION OF CRIME:  ASSAULT/BATTERY","COMMISSION OF CRIME:  MURDER/MANSLAUGHTER ETC","ARREST/LOCKUP PROCEDURE:  PROPER CARE/INJURY/DEATH"]
duplicate_words_crime=["EXCESSIVE FORCE:  DOMESTIC"]

#identify duplicate words here 
#cmp[["civ_complaint_flag","civ_complaint_force","civ_complaint_detain","civ_complaint_hate","civ_complaint_crime"]].value_counts()
#cmp.loc[(cmp["civ_complaint_force"]==True) & (cmp["civ_complaint_detain"]==True)]["civ_complaint_desc"].value_counts()

for word in duplicate_words_force:
    cmp["civ_complaint_force"]=np.where(cmp["civ_complaint_desc"].str.contains(word,case=False),False,cmp["civ_complaint_force"])
for word in duplicate_words_detain:
    cmp["civ_complaint_detain"]=np.where(cmp["civ_complaint_desc"].str.contains(word,case=False),False,cmp["civ_complaint_detain"])
for word in duplicate_words_crime:
    cmp["civ_complaint_crime"]=np.where(cmp["civ_complaint_desc"].str.contains(word,case=False),False,cmp["civ_complaint_crime"])


In [372]:
cmp["comp_cat"]=np.where(cmp.final_finding.isna(),"Missing",cmp.final_finding)
cmp["comp_cat"]=cmp["comp_cat"].replace(comp_cat)

cmp["dept_complaint_flag"]=np.where(cmp["CITIZEN / DEPT"]=="DEPT",True,False)
cmp["dept_complaint_cleaned_flag"]=np.where(cmp["CITIZEN / DEPT"]=="DEPT",np.where(cmp.complaint_category.isin(dept_exclude),False,True),False)
cmp["dept_complaint_desc"]=np.where(cmp["dept_complaint_cleaned_flag"],cmp.complaint_category,"")

cmp["civ_complaint_cat"]=np.where(cmp.civ_complaint_force,"FORCE",
    np.where(cmp.civ_complaint_detain,"DETAIN",
    np.where(cmp.civ_complaint_hate,"HATE",
    np.where(cmp.civ_complaint_crime,"CRIME",
    np.where(cmp.civ_complaint_flag,"OTHER","")))))
    
#checking flag made correctly
#cmp[cmp.dept_complaint_flag==1][["complaint_category","dept_complaint_cleaned_flag"]].value_counts()

In [373]:
cmp_counts=cmp.groupby("UID",as_index=False).agg({"dept_complaint_flag":"sum","dept_complaint_cleaned_flag":"sum","civ_complaint_flag":"sum","civ_complaint_force":"sum","civ_complaint_detain":"sum","civ_complaint_hate":"sum","civ_complaint_crime":"sum"})
df=df.merge(cmp_counts,on="UID",how="left")

flags=["dept_complaint_cleaned_flag","civ_complaint_flag","civ_complaint_force","civ_complaint_detain","civ_complaint_hate","civ_complaint_crime"]
col_names=["dept_complaint_desc","civ_complaint_desc","civ_complaint_force_desc","civ_complaint_detain_desc","civ_complaint_hate_desc","civ_complaint_crime_desc"]
for i, flag in enumerate(flags):
    temp=cmp[cmp[flag]==True].groupby("UID")["complaint_category"].apply(list).reset_index()
    temp.columns=["UID",col_names[i]]
    df=df.merge(temp,on="UID",how="left")


## TTR

In [374]:
#merge trr
trr_overall=pd.read_csv("TRR-main_2004-2016_2016-09.csv")
trr_subjects=pd.read_csv("TRR-subjects_2004-2016_2016-09.csv")
trr_officers=pd.read_csv("TRR-officers_2004-2016_2016-09.csv")
trr_actions=pd.read_csv("TRR-actions-responses_2004-2016_2016-09.csv")

trr=trr_overall.merge(trr_officers,on="trr_id",suffixes=("","_officer")) #,how="left"
trr=trr.merge(trr_subjects,on="trr_id",suffixes=("","_subject")) #,how="left"
trr=trr.merge(trr_actions[trr_actions.person=="Member Action"],on="trr_id",suffixes=("","_subject")) #,how="left"


trr["trr_year"]=trr["trr_date"].apply(lambda x: int(str(x)[0:4]))
trr_14=trr[trr["trr_year"]==2014]


In [375]:
action_cat={"Missing":"Missing","Verbal Commands":"Other","Member Presence":"Other","Physical Force - Stunning":"Use of Force","Physical Force - Holding":"Use of Force","Physical Force - Direct Mechanical":"Use of Force","Other Force":"Use of Force","Taser":"Use of Weapon","Chemical":"Use of Weapon","Impact Weapon":"Use of Weapon","Firearm":"Use of Weapon","Taser Display":"Other","Chemical (Authorized)":"Use of Weapon","Other":"Other"} 

In [376]:
trr_14["force_cat"]=np.where(trr_14.force_type.isna(),"Missing",trr_14.force_type.replace(action_cat))
trr_14["flag"]=True
trr_14["weapon_flag"]=np.where(trr_14["force_cat"]=="Use of Weapon",True,False)
trr_14["force_flag"]=np.where(trr_14["force_cat"]=="Use of Force",True,trr_14["weapon_flag"])



C:\Users\Olivia_Pinney\AppData\Local\Temp\ipykernel_17176\3391038428.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trr_14["force_cat"]=np.where(trr_14.force_type.isna(),"Missing",trr_14.force_type.replace(action_cat))
C:\Users\Olivia_Pinney\AppData\Local\Temp\ipykernel_17176\3391038428.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trr_14["flag"]=True
C:\Users\Olivia_Pinney\AppData\Local\Temp\ipykernel_17176\3391038428.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

In [377]:
trr_14_counts=trr_14.groupby("UID",as_index=False).agg({"force_flag":"sum","weapon_flag":"sum"})
    
df=df.merge(trr_14_counts,on="UID",how="left")

flags=["force_flag","weapon_flag"]
col_names=["action_force_desc","action_weapon_desc"]
for i, flag in enumerate(flags):
    temp=trr_14[trr_14[flag]==True].groupby("UID")["action"].apply(list).reset_index()
    temp.columns=["UID",col_names[i]]
    df=df.merge(temp,on="UID",how="left")


In [378]:
temp

,UID,action_weapon_desc
0,100010.0,[TASER (PROBE DISCHARGE)]
1,100036.0,[TASER (PROBE DISCHARGE)]
2,100068.0,"[TASER (PROBE DISCHARGE), TASER (PROBE DISCHAR..."
3,100077.0,[TASER (PROBE DISCHARGE)]
4,100200.0,[TASER (PROBE DISCHARGE)]
...,...,...
521,132258.0,[O.C./CHEMICAL WEAPON]
522,132269.0,[IMPACT WEAPON (DESCRIBE IN ADDITIONAL INFO)]
523,132277.0,[TASER (PROBE DISCHARGE)]
524,132328.0,[FIREARM]


# Export

In [379]:
col_names=["force_flag","weapon_flag","dept_complaint_flag","dept_complaint_cleaned_flag","civ_complaint_flag","civ_complaint_force","civ_complaint_detain","civ_complaint_hate","civ_complaint_crime"]

for col in col_names:
    df[col+"_persons_with"]=np.where(df[col]>0,1,0)

In [380]:
unit_all=df.groupby("unit_description").agg({"UID":"count","force_flag":"sum","weapon_flag":"sum","dept_complaint_flag":"sum","dept_complaint_cleaned_flag":"sum","civ_complaint_flag":"sum","civ_complaint_force":"sum","civ_complaint_detain":"sum","civ_complaint_hate":"sum","civ_complaint_crime":"sum","force_flag_persons_with":"sum","weapon_flag_persons_with":"sum","dept_complaint_flag_persons_with":"sum","dept_complaint_cleaned_flag_persons_with":"sum","civ_complaint_flag_persons_with":"sum","civ_complaint_force_persons_with":"sum","civ_complaint_detain_persons_with":"sum","civ_complaint_hate_persons_with":"sum","civ_complaint_crime_persons_with":"sum"})
unit_cats=df.groupby("unit_cat").agg({"UID":"count","force_flag":"sum","weapon_flag":"sum","dept_complaint_flag":"sum","dept_complaint_cleaned_flag":"sum","civ_complaint_flag":"sum","civ_complaint_force":"sum","civ_complaint_detain":"sum","civ_complaint_hate":"sum","civ_complaint_crime":"sum","force_flag_persons_with":"sum","weapon_flag_persons_with":"sum","dept_complaint_flag_persons_with":"sum","dept_complaint_cleaned_flag_persons_with":"sum","civ_complaint_flag_persons_with":"sum","civ_complaint_force_persons_with":"sum","civ_complaint_detain_persons_with":"sum","civ_complaint_hate_persons_with":"sum","civ_complaint_crime_persons_with":"sum"})

unit_all.to_json("departments.json",orient="records")
unit_all.to_json("departments_cats.json",orient="records")

In [381]:
cols_to_keep=['gender', 'race', 'birth_year', 'current_age','appointed_date', 'current_rank', 'unit_description', 'first_name', 'last_name', 'middle_initial', 'suffix_name','UID','dept_complaint_cleaned_flag', 'civ_complaint_flag','civ_complaint_force', 'civ_complaint_detain', 'civ_complaint_hate','civ_complaint_crime', 'dept_complaint_desc', 'civ_complaint_desc','civ_complaint_force_desc', 'civ_complaint_detain_desc','civ_complaint_hate_desc', 'civ_complaint_crime_desc', 'force_flag','weapon_flag', 'action_force_desc', 'action_weapon_desc']
df[cols_to_keep].to_json("officers_2014.json",orient="records")


In [382]:
#df.force_flag.value_counts()